<a href="https://colab.research.google.com/github/Djensonsan/Information_Retrieval_Assignment_2/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval Assignment 2: LDA


## Runtime specs

In [20]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [21]:
!cat /proc/meminfo

MemTotal:       26751732 kB
MemFree:        18012160 kB
MemAvailable:   23430000 kB
Buffers:          107868 kB
Cached:          3907396 kB
SwapCached:            0 kB
Active:          3011532 kB
Inactive:        5342720 kB
Active(anon):    2703760 kB
Inactive(anon):      396 kB
Active(file):     307772 kB
Inactive(file):  5342324 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               376 kB
Writeback:             0 kB
AnonPages:       4339000 kB
Mapped:           297068 kB
Shmem:               940 kB
Slab:             239544 kB
SReclaimable:     181044 kB
SUnreclaim:        58500 kB
KernelStack:        5008 kB
PageTables:        14444 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13375864 kB
Committed_AS:    7714980 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1904 kB
AnonHugePages:   

## Imports

In [25]:
# Install your required packages here
!pip install pandas numpy matplotlib fsspec gcsfs dask[complete] gensim nltk
!pip install -q tqdm

In [26]:
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import dask.dataframe as dd

from ast import literal_eval
import gensim
from gensim import corpora, models
from tqdm import tqdm
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
# Mount google drive in colab:
from google.cloud import storage
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing

In [28]:
# Pycharm:
# data = pd.read_csv('data/news_dataset.csv')

In [29]:
# Colab:
data = pd.read_csv('/content/drive/MyDrive/IR-Assignment-2/data/news_dataset.csv')

### Exploration

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141585 entries, 0 to 141584
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           141585 non-null  int64 
 1   title        141583 non-null  object
 2   publication  141564 non-null  object
 3   author       125725 non-null  object
 4   date         138869 non-null  object
 5   year         138911 non-null  object
 6   month        138911 non-null  object
 7   url          84541 non-null   object
 8   content      141543 non-null  object
dtypes: int64(1), object(8)
memory usage: 9.7+ MB


In [31]:
data.head(n=43)

,id,title,publication,author,date,year,month,url,content
0,1,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016,12,NaN,WASHINGTON — Congressional Republicans have...
1,2,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017,6,NaN,"After the bullet shells get counted, the blood..."
2,3,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017,1,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,4,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017,4,NaN,"Death may be the great equalizer, but it isn’t..."
4,5,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017,1,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
5,6,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,2017,1,NaN,"LONDON — Queen Elizabeth II, who has been b..."
6,7,Taiwan’s President Accuses China of Renewed In...,New York Times,Javier C. Hernández,2017-01-02,2017,1,NaN,BEIJING — President Tsai of Taiwan sharpl...
7,8,"After ‘The Biggest Loser,’ Their Bodies Fought...",New York Times,Gina Kolata,2017-02-08,2017,2,NaN,"Danny Cahill stood, slightly dazed, in a blizz..."
8,9,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,Katherine Rosman,2016-12-31,2016,12,NaN,"Just how is Hillary Kerr, the founder of ..."
9,10,Calling on Angels While Enduring the Trials of...,New York Times,Andy Newman,2016-12-31,2016,12,NaN,Angels are everywhere in the Muñiz family’s ap...


In [32]:
# Use document 42 as running example
data.loc[42, 'content']

'WASHINGTON  —     Donald J. Trump on Tuesday named as his chief trade negotiator a Washington lawyer who has long advocated protectionist policies, the latest sign that Mr. Trump intends to fulfill his campaign promise to get tough with China, Mexico and other trading partners. Mr. Trump also renewed his episodic campaign to persuade American companies to expand domestic manufacturing, criticizing General Motors via Twitter on Tuesday morning for making in Mexico some of the Chevrolet Cruze hatchbacks it sells domestically. Hours later, Mr. Trump claimed credit after Ford said it would expand vehicle production in Flat Rock, Mich. The choice of Robert Lighthizer (pronounced   ) to be the United States’ trade representative nearly completes Mr. Trump’s selection of top economic advisers and, taken together with the  ’s running commentary on Twitter, underscores Mr. Trump’s focus on making things in America. That is causing unease among some Republicans who regard Mr. Trump’s views on t

### Keep document content

In [33]:
data_content = data['content']

In [34]:
type(data_content)

pandas.core.series.Series

In [35]:
data_content.head(n=3)

0    WASHINGTON  —   Congressional Republicans have...
1    After the bullet shells get counted, the blood...
2    When Walt Disney’s “Bambi” opened in 1942, cri...
Name: content, dtype: object

### Tokenization, Stemming and Lemmatization

In [36]:
tqdm.pandas()
# Note Jens: Might want to use Dask to speed things up.
# When using Dask can't use tqdm as far as I know.

In [37]:
# There's NaN values in the dataset
data_content.dropna(inplace=True)

In [38]:
data_content.isna().any()

False

In [39]:
# Tokenization
data_content_tokenized = data_content.progress_apply(lambda x: nltk.word_tokenize(x))

100%|██████████| 141543/141543 [14:40<00:00, 160.69it/s]


In [40]:
# Remove words smaller than 3 characters
data_content_tokenized = data_content_tokenized.progress_apply(lambda x: [y for y in x if len(y)>2])

100%|██████████| 141543/141543 [00:17<00:00, 8317.89it/s]


In [41]:
# Remove Stopswords
stop_words = set(stopwords.words('english')) 
data_content_stopped = data_content_tokenized.progress_apply(lambda x: [y for y in x if not y in stop_words])

100%|██████████| 141543/141543 [00:14<00:00, 9769.42it/s] 


In [42]:
# Stemming and Lemmatization 
stemmer = SnowballStemmer("english")
data_content_clean = data_content_stopped.progress_apply(lambda x: [stemmer.stem(WordNetLemmatizer().lemmatize(y)) for y in x])

100%|██████████| 141543/141543 [17:54<00:00, 131.79it/s]


In [43]:
data_content_clean.head()

0    [washington, congression, republican, new, fea...
1    [after, bullet, shell, get, count, blood, dri,...
2    [when, walt, disney, bambi, open, 1942, critic...
3    [death, may, great, equal, necessarili, evenha...
4    [seoul, south, korea, north, korea, leader, ki...
Name: content, dtype: object

In [44]:
# data_content_clean contain the cleaned 'content' column of the news dataset:
data_content_clean.to_csv('/content/drive/MyDrive/IR-Assignment-2/data/new_dataset_clean.csv')

In [ ]:
data_content_stopped = data_content_clean.progress_apply(lambda x: [y for y in x if y in stop_words])

# LDA

The following part will use Python libraries for performing an LDA. The result can be used for comparison with our model.

The code follows a tutorial:
[LDA Tutorial](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24)

In [ ]:
data_content_clean = pd.read_csv('/content/drive/MyDrive/IR-Assignment-2/data/new_dataset_clean.csv')

In [ ]:
data_content_clean = data_content_clean['content']

In [ ]:
data_content_clean.head()

0    ['washington', 'congression', 'republican', 'n...
1    ['bullet', 'shell', 'get', 'count', 'blood', '...
2    ['walt', 'disney', 'bambi', 'open', '1942', 'c...
3    ['death', 'may', 'great', 'equal', 'necessaril...
4    ['seoul', 'south', 'korea', 'north', 'korea', ...
Name: content, dtype: object

In [ ]:
dictionary = gensim.corpora.Dictionary()
for article in tqdm(data_content_clean):
  dictionary.add_documents([literal_eval(article)])

100%|██████████| 141543/141543 [03:13<00:00, 730.26it/s]


## Sanity check

In [ ]:
print(dictionary)

Dictionary(274259 unique tokens: ['2010', '2015', '2017', 'access', 'acknowledg']...)


In [ ]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 2010
1 2015
2 2017
3 access
4 acknowledg
5 act
6 administr
7 advoc
8 afford
9 alli
10 american


In [ ]:
# Filter out tokens that appear in less than 15 documents, more than 0.5 documents (fraction of total corpus size, not absolute number).
# keep only the first 100000 most frequent tokens.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

## doc2bow

For each document we create a dictionary reporting how many words and how many times those words appear.

In [ ]:
bow_corpus = [dictionary.doc2bow(literal_eval(doc)) for doc in data_content_clean]
bow_corpus[42]

[(1, 1),
 (3, 1),
 (6, 5),
 (7, 3),
 (8, 1),
 (10, 10),
 (13, 1),
 (26, 1),
 (35, 2),
 (36, 1),
 (47, 1),
 (52, 1),
 (53, 2),
 (55, 1),
 (58, 1),
 (60, 1),
 (62, 1),
 (63, 1),
 (67, 1),
 (71, 1),
 (74, 1),
 (78, 1),
 (80, 1),
 (85, 1),
 (99, 2),
 (101, 2),
 (115, 3),
 (117, 1),
 (124, 2),
 (128, 6),
 (136, 2),
 (138, 1),
 (139, 2),
 (141, 1),
 (144, 1),
 (145, 1),
 (147, 1),
 (148, 2),
 (150, 3),
 (151, 2),
 (154, 1),
 (158, 1),
 (162, 1),
 (163, 1),
 (165, 1),
 (170, 1),
 (171, 32),
 (172, 3),
 (175, 1),
 (176, 2),
 (184, 1),
 (201, 1),
 (202, 1),
 (205, 1),
 (212, 1),
 (216, 1),
 (219, 3),
 (225, 5),
 (227, 1),
 (231, 1),
 (232, 3),
 (234, 2),
 (235, 2),
 (236, 1),
 (238, 1),
 (241, 1),
 (247, 5),
 (251, 1),
 (259, 3),
 (266, 1),
 (270, 23),
 (274, 4),
 (277, 2),
 (278, 2),
 (281, 1),
 (282, 3),
 (283, 1),
 (285, 2),
 (313, 1),
 (314, 1),
 (319, 4),
 (332, 1),
 (333, 1),
 (334, 1),
 (338, 2),
 (348, 1),
 (374, 1),
 (376, 1),
 (403, 1),
 (411, 2),
 (427, 2),
 (429, 2),
 (443, 3),
 (44

In [ ]:
bow_doc_42 = bow_corpus[42]
for i in range(len(bow_doc_42)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_42[i][0],
                                               dictionary[bow_doc_42[i][0]],
bow_doc_42[i][1]))

Word 1 ("2015") appears 1 time.
Word 3 ("access") appears 1 time.
Word 6 ("administr") appears 5 time.
Word 7 ("advoc") appears 3 time.
Word 8 ("afford") appears 1 time.
Word 10 ("american") appears 10 time.
Word 13 ("anoth") appears 1 time.
Word 26 ("big") appears 1 time.
Word 35 ("case") appears 2 time.
Word 36 ("caus") appears 1 time.
Word 47 ("committe") appears 1 time.
Word 52 ("congress") appears 1 time.
Word 53 ("congression") appears 2 time.
Word 55 ("conserv") appears 1 time.
Word 58 ("consum") appears 1 time.
Word 60 ("continu") appears 1 time.
Word 62 ("cost") appears 1 time.
Word 63 ("could") appears 1 time.
Word 67 ("deal") appears 1 time.
Word 71 ("demand") appears 1 time.
Word 74 ("despit") appears 1 time.
Word 78 ("discuss") appears 1 time.
Word 80 ("disput") appears 1 time.
Word 85 ("donald") appears 1 time.
Word 99 ("even") appears 2 time.
Word 101 ("execut") appears 2 time.
Word 115 ("general") appears 3 time.
Word 117 ("govern") appears 1 time.
Word 124 ("hous") app

## TF-IDF

In [ ]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    print(doc)
    break

[(0, 0.0297297121835003), (1, 0.020115641857962285), (2, 0.027475091478599733), (3, 0.0246338131895793), (4, 0.027728042071150663), (5, 0.0174134067651629), (6, 0.2237821409045775), (7, 0.0293950978322572), (8, 0.029892524774677208), (9, 0.026425743525530152), (10, 0.011398116803114852), (11, 0.03411990990197013), (12, 0.02989519358859514), (13, 0.012361490118588869), (14, 0.07765756515012612), (15, 0.10685349563361013), (16, 0.09972071792922374), (17, 0.027009297370733168), (18, 0.03654156669594325), (19, 0.034443188129025), (20, 0.034751792421216156), (21, 0.025956906486448782), (22, 0.04380777945480366), (23, 0.04118632475193582), (24, 0.009395295832206408), (25, 0.03613409224875905), (26, 0.017048529185535587), (27, 0.04649347886775031), (28, 0.09265695278345015), (29, 0.058827631973247335), (30, 0.18737662261388102), (31, 0.03161267072451408), (32, 0.03760461316546215), (33, 0.1485736490401198), (34, 0.0603689586214145), (35, 0.05980911701364772), (36, 0.03915834760553407), (37, 0

## LDA Library: gensim.models.LdaMulticore

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.013*"child" + 0.013*"drug" + 0.013*"famili" + 0.008*"life" + 0.008*"mother" + 0.006*"parent" + 0.006*"medic" + 0.006*"doctor" + 0.006*"get" + 0.006*"day"
Topic: 1 
Words: 0.026*"polic" + 0.015*"offic" + 0.009*"report" + 0.008*"gun" + 0.008*"kill" + 0.007*"shoot" + 0.007*"told" + 0.006*"charg" + 0.006*"investig" + 0.005*"man"
Topic: 2 
Words: 0.092*"trump" + 0.017*"presid" + 0.013*"donald" + 0.007*"campaign" + 0.007*"obama" + 0.006*"white" + 0.005*"first" + 0.004*"call" + 0.004*"washington" + 0.004*"presidenti"
Topic: 3 
Words: 0.012*"forc" + 0.011*"state" + 0.010*"militari" + 0.010*"attack" + 0.009*"islam" + 0.008*"syria" + 0.007*"group" + 0.006*"govern" + 0.006*"syrian" + 0.006*"offici"
Topic: 4 
Words: 0.013*"think" + 0.011*"get" + 0.010*"thing" + 0.010*"know" + 0.010*"becaus" + 0.009*"want" + 0.009*"make" + 0.009*"way" + 0.009*"go" + 0.007*"realli"
Topic: 5 
Words: 0.008*"show" + 0.006*"film" + 0.004*"music" + 0.004*"movi" + 0.004*"play" + 0.004*"stori" + 0.004*"b

## Evaluation

In [ ]:
# Let's look at a test document: document 42
# 1) clearly see that the article is about the economy 40%
# 2) US politics 27%
# 3) US Justice Department 8%
# 4) US Congress 6%
for index, score in sorted(lda_model[bow_corpus[42]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.43498122692108154	 
Topic: 0.014*"percent" + 0.009*"tax" + 0.008*"plan" + 0.006*"rate" + 0.006*"health" + 0.006*"state" + 0.005*"bank" + 0.005*"polici" + 0.005*"million" + 0.005*"trump"

Score: 0.17876674234867096	 
Topic: 0.092*"trump" + 0.017*"presid" + 0.013*"donald" + 0.007*"campaign" + 0.007*"obama" + 0.006*"white" + 0.005*"first" + 0.004*"call" + 0.004*"washington" + 0.004*"presidenti"

Score: 0.08457224071025848	 
Topic: 0.026*"china" + 0.020*"north" + 0.017*"korea" + 0.012*"nuclear" + 0.010*"south" + 0.010*"chines" + 0.010*"state" + 0.009*"unit" + 0.009*"missil" + 0.007*"countri"

Score: 0.07658499479293823	 
Topic: 0.022*"court" + 0.017*"law" + 0.017*"state" + 0.010*"rule" + 0.010*"feder" + 0.010*"case" + 0.008*"justic" + 0.008*"right" + 0.008*"judg" + 0.007*"legal"

Score: 0.05687502399086952	 
Topic: 0.018*"trump" + 0.013*"presid" + 0.012*"investig" + 0.011*"hous" + 0.011*"offici" + 0.010*"report" + 0.007*"fbi" + 0.007*"depart" + 0.007*"intellig" + 0.007*"white"

S

In [ ]:
# Decide the most representative documents for each topic and write it to file

repList = [[] for i in range(20)]

for doc_id in tqdm(range(len(data_content_clean)), "Scoring: "):
  for index, score in sorted(lda_model[bow_corpus[doc_id]], key=lambda tup: -1*tup[1]):
    repList[index].append((doc_id, score))

for topic in repList:
  topic.sort(key=lambda x:x[1], reverse=True)

topicList = ["Topic "+str(i) for i in range(20)]

df = pd.DataFrame(columns=topicList)

for topic in range(len(topicList)):
  end = 100
  if len(repList[topic]) < 100:
    end = len(repList[topic])-1
  df["Topic "+ str(topic)] = [x[0] for x in repList[topic][0:end]]

df.to_csv('/content/drive/MyDrive/IR-Assignment-2/data/topic_document_rank_library.csv')


Scoring: 100%|██████████| 141543/141543 [03:52<00:00, 609.00it/s]


In [ ]:
file = open("/content/drive/MyDrive/IR-Assignment-2/data/analysis_topic_per_doc_lib.txt","w") 

for i in range(10):
  file.write("doc " + str(i) + " topics: \n")
  for index, score in sorted(lda_model[bow_corpus[i]], key=lambda tup: -1*tup[1]):
    file.write(str(score) + "   ")
  file.write("\n\n")

file.close()